In [0]:
# Use with associated Data Lake Storage with Azure Databricks -dbfs
# dbfs:/FileStore/shared_uploads/iriscloudone@outlook.com/greentaxi.parquet

# Use with External Data Lake Storage with Azure Datarbricks - abfss
# abfss://datalakefs@ssadcloudazsynapsedl2.dfs.core.windows.net/greentaxi/greentaxi.parquet


In [0]:
# Assign with your Data Lake Values
storage_account_name = 'ssadcloudazsynapsedl2'
container_name = 'datalakefs'
path_to_data = 'greentaxi/greentaxi.parquet'
storage_account_access_key = 'key'

In [0]:
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_access_key
)

In [0]:
abfss_greentaxi_file_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{path_to_data}"

In [0]:
print(abfss_greentaxi_file_path)

In [0]:
greentaxiDF = spark.read.parquet(abfss_greentaxi_file_path)

In [0]:
#df.write.format("delta").mode("overwrite").save("/tmp/delta/people10m")

In [0]:
curatedgreentaxiDF = greentaxiDF['VendorID','lpep_pickup_datetime','lpep_dropoff_datetime','passenger_count','fare_amount']

In [0]:
display(curatedgreentaxiDF)

In [0]:
# Write to Delta Format
#df.write.format("delta").mode("overwrite").save("/tmp/delta/people10m")
curatedgreentaxiDF.write.format("delta").mode("overwrite").save("/tmp/delta/greentaxi")

In [0]:
# Create Delta Table
curatedgreentaxiDF.write.format("delta").mode("overwrite").saveAsTable("curated_greentaxi")

In [0]:
FareAmountbyPassengercountQuery = 'SELECT SUM(fare_amount) as FareAmountbyPassengercount, passenger_count FROM curated_greentaxi GROUP BY passenger_count ORDER BY passenger_count'

In [0]:
FareAmountbyPassengercountResults = spark.sql(FareAmountbyPassengercountQuery)

In [0]:
display(FareAmountbyPassengercountResults)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
CREATE DATABASE NewYorkGreenTaxiDB

In [0]:
%sql
USE DATABASE NewYorkGreenTaxiDB

In [0]:
%sql
CREATE TABLE greentaxiTable

In [0]:
%sql
ALTER TABLE greentaxiTable 
SET TBLPROPERTIES ('delta.feature.timestampNtz' = 'supported');

In [0]:
%sql
COPY INTO greentaxiTable
FROM 'abfss://datalakefs@ssadcloudazsynapsedl2.dfs.core.windows.net/greentaxi/greentaxi.parquet'
FILEFORMAT = Parquet
COPY_OPTIONS ('mergeSchema' = 'true');